In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch, datasets

model_name = "facebook/opt-350m"
tokenizer   = AutoTokenizer.from_pretrained(model_name)

# ★ポイント: num_labels=6
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=6,
    torch_dtype=torch.float16,
    quantization_config={"load_in_4bit": True}
)

# LoRA アダプタ
lora_cfg = LoraConfig( task_type="SEQ_CLS", r=16, lora_alpha=16, bias="none")
model = get_peft_model(base_model, lora_cfg)
model.config.use_cache = False

def preprocess(ex):
    enc = tokenizer(ex["prompt"] + ex["idea"], truncation=True, max_length=512)
    enc["labels"] = [ex[f"score_{i}"] for i in range(6)]  # 6 次元ラベル
    return enc

ds = datasets.load_dataset("your_org/irm_dataset")  # 自前データ
ds = ds.map(preprocess)

args = TrainingArguments(
    output_dir="./irm_logs",
    per_device_train_batch_size=4,
    learning_rate=2e-5,
    num_train_epochs=5,
    logging_steps=20,
    save_steps=200
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["val"]
)

trainer.train()
model.save_pretrained("./irm_reward_model")